<h1>Set 2</h1>
<h5>Challenge 9</h5>
<p>Implement PKCS#7 padding</p>

In [ ]:
from crypto import *

plaintext = "YELLOW SUBMARINE"
result = "YELLOW SUBMARINE\x04\x04\x04\x04"
pkcs7padding(plaintext, 20) == result

<h5>Challenge 10</h5>
<p>Implement CBC</p>

In [ ]:
with open("files/2_10.txt", 'r') as f:
    data = f.read()

In [ ]:
key = b'YELLOW SUBMARINE'
iv = b'\x00\x00\x00 &c'
cbc = CBC(key = key, iv = iv)
print(cbc.decrypt(base642bytes(data)))

<h5>Challenge 11</h5>
<p>ECB/CBC detection oracle</p>

In [ ]:
from crypto import *
for i in range(10):
    oracle = EncryptionOracle("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
    print("Guesser:", oracle.guessMode() ,'Correct:',oracle.getMode())

<h5>Challenge 12</h5>
<p>Byte-at-a-time ECB decryption</p>

In [ ]:
data = base642bytes('''Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK''')

cracker = ECBCracker()
cracker.findBlocksize()
print(cracker.crack(data))

In [ ]:
# challenge 13

In [ ]:
from textoperations import *
url = 'foo=bar&baz=qux&zap=zazzle'
decodeURLParams(url)

In [ ]:
profileFor("foo@bar.com")

In [ ]:
# sanity check
profileFor("foo@bar.com&role=admin")

In [ ]:
# random aes key
from crypto import *
key = getRandomBytes(16)
key

In [ ]:
cipher = Cipher(key)
ciphertext = cipher.encryptECB(profileFor("foo@bar.com"))
ciphertext

In [ ]:
cipher.decryptECB(ciphertext)

In [ ]:
# using only input to profileFor & operations on ciphertext to create role=admin
# find out length of mail to isolate role, user has 4 letters, therefore to have 
# it in a single block make mail length of block + 4
# -> 13

In [ ]:
for i in range(5,11):
    print("LenMail:", i, "LenCiphertext:" ,len(cipher.encryptECB(profileFor('x' * i))))

In [ ]:
 ciphertext = cipher.encryptECB(profileFor('x' * 13))

In [ ]:
cipher.decryptECB(ciphertext)

In [ ]:
len(ciphertext[:16])

In [ ]:
print(cipher.decryptECB(ciphertext[32:]))
len(ciphertext[32:])

In [ ]:
# next i need a valid padded block of admin
pkcs7padding('admin')

In [ ]:
# since 'email=' is before, i need to add 10 signs in front
pad = 'x' * 10
ciphertext2 = cipher.encryptECB(profileFor(pad + pkcs7padding('admin')))

In [ ]:
cipher.decryptECB(ciphertext2[16:32])

In [ ]:
front = ciphertext[:32]
front

In [ ]:
back = ciphertext2[16:32]
back

In [ ]:
cipher.decryptECB(front + back)

In [ ]:
  from crypto import *

In [ ]:
test1 = "ICE ICE BABY\x04\x04\x04\x04"
test2 = "ICE ICE BABY\x05\x05\x05\x05"
test3 = "ICE ICE BABY\x01\x02\x03\x04"

In [ ]:
removepkcs7padding(test1)

In [ ]:
removepkcs7padding(test2)

In [ ]:
removepkcs7padding(test3)

In [1]:
# challenge 14
from crypto import *
from random import randint

In [20]:
paddingFront = randint(1,10)
bytesFront = getRandomBytes(paddingFront)
data = base642bytes('''Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK''')

In [78]:
cracker = ECBCracker()
cracker.findBlocksize()

In [8]:
# padding in front does not really matter if it can be isolated
# therefore "finding" the size of bytes at front
# then completly isolating them with RRRR RRRx xxxx xxxx // xxxx xxxx xxxx xxxx 
# and removing first block

In [ ]:
# i can find out the size of the padding by adding a lot of x and check until the first block
# does not change anymore, which indicates, that they reach now into the 3rd block

In [24]:
lastBlock = getRandomBytes(16)
guessedPadLen = -1
for i in range(16):
    tmpCiphertext = cracker.encryptWithPadding(bytesFront, b'x' * i, data)[:16]
    if tmpCiphertext == lastBlock:
        guessedPadLen = 16 - i + 1
        print("Stopped changing at len:", i)
        print("Random number is:", guessedPadLen)
        break
    else:
        lastBlock = tmpCiphertext

Stopped changing at len: 10
Random number is: 7


In [38]:
# now that i know the size of the random strings, i can just fill first block & second block 
# and do as before
plaintext = []

# buffer of known chars
buffer = b'x' * (15) + b'x' * (16 - guessedPadLen)

# make char dict
charDict = cracker.makeCharDict()

# len of message 
msgSize = len(cracker.encryptWithPadding(bytesFront, b'', data)) - guessedPadLen

In [77]:
plaintext = []
for i in range(msgSize):
    toEncrypt = (bytesFront +  buffer + bytes([data[i]]))[16:32]
    tmpCiphertext = cracker.cipher.encryptBlockECB(toEncrypt)
    plaintext.append(charDict[tmpCiphertext])
print(b''.join(plaintext).decode('utf-8'))
    
    

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



In [64]:
len(data)

138

In [65]:
msgSize = len(cracker.encrypt(bytesFront, data)) - guessedPadLen
msgSize

153

In [71]:
# find length of data 
start = 16-guessedPadLen
end = start +16
fullSize = len(cracker.encryptWithPadding(bytesFront, b'', data))
msgSize = -1
last = len(cracker.encryptWithPadding(bytesFront, b'x' * start, data))
for i in range(start + 1, end):
    if len(cracker.encryptWithPadding(bytesFront, b'x' * i, data)) != last:
        msgSize = fullSize - i - guessedPadLen + 1
        break
print(msgSize)

138


In [55]:
# len of data -> 153 - 15
fullSize

160